In [79]:
import os
import numpy as np

import tensorflow as tf

from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Conv2D, Conv2DTranspose, Reshape, BatchNormalization
from tensorflow.keras import optimizers, losses
from tensorflow.keras.datasets import mnist

import matplotlib.pyplot as plt

In [80]:
os.listdir()

['.git',
 '.gitignore',
 '.ipynb_checkpoints',
 'autoencoder.ipynb',
 'datasets',
 'full_numpy_bitmap_angel.npy',
 'GANs.ipynb',
 'input_data.csv',
 'prepare_data.ipynb',
 'seq2seq_model.ipynb',
 'text_summary.ipynb']

In [115]:
t = np.load('full_numpy_bitmap_angel.npy')
x_train = []
for x in t:
    x_train.append(x.reshape(28, 28, 1))
x_train = np.array(x_train)
x_train = np.float32(x_train)

In [116]:
class Discriminator(Model):
    def __init__(self, **kwargs):
        super(Discriminator, self).__init__(**kwargs)
        self.dense_1 = Dense(64, activation='relu')
        self.dense_2 = Dense(32, activation='relu')
        self.dense_3 = Dense(32, activation='relu')
        self.flat_layer = Flatten()
        self.output_layer = Dense(1, activation='sigmoid')
        
    def call(self, inputs):
        x = self.dense_1(inputs)
        x = self.dense_2(x)
        x = self.dense_3(x)
        x = self.flat_layer(x)
        output = self.output_layer(x)
        return output
    
    
class Generator(Model):
    def __init__(self, **kwargs):
        super(Generator, self).__init__(**kwargs)
        self.dense_1 = Dense(7*7*256, activation='relu', input_shape=(100,), name='dense_1')
        self.reshape = Reshape((7, 7, 256), name='reshape')
        
        self.conv2dt_1 = Conv2DTranspose(128, (3, 3), strides=1, padding='same', name='conv2dt_1')
        self.bn1 = BatchNormalization(name='bn1')
        
        self.conv2dt_2 = Conv2DTranspose(64, (3, 3), strides=2, padding='same', name='conv2dt_2')
        self.bn2 = BatchNormalization(name='bn2')
        
        self.conv2dt_3 = Conv2DTranspose(1, (3, 3), strides=2, padding='same', 
                                         activation='tanh', name='conv2dt_3')
        
    def call(self, inputs):
        x = self.dense_1(inputs)
        x = self.reshape(x)
        x = self.conv2dt_1(x)
        x = self.bn1(x)
        x = self.conv2dt_2(x)
        x = self.bn2(x)
        output = self.conv2dt_3(x)
        return output
    
    
class GAN(Model):
    def __init__(self, latent_dim, **kwargs):
        super(GAN, self).__init__(**kwargs)
        self.discriminator = Discriminator()
        self.generator = Generator()
        self.latent_dim = latent_dim
        
    def compile(self, d_optimizer, g_optimizer, loss_fn):
        super(GAN, self).compile()
        self.d_optimizer = d_optimizer
        self.g_optimizer = g_optimizer
        self.loss_fn = loss_fn
    
    def train_step(self, data):
        batch_size = tf.shape(data)[0]
        fake_img = tf.random.normal(shape=(batch_size, self.latent_dim))
        fake_img = self.generator(fake_img)
        both_img = tf.concat([fake_img, data], axis=0)
        labels = tf.concat([tf.ones((batch_size, 1)),
                            tf.zeros((batch_size, 1))],
                            axis=0)
        labels += 0.05 * tf.random.uniform(shape=(2 * batch_size, 1))
        with tf.GradientTape() as tape:
            preds = self.discriminator(both_img)
            d_loss = self.loss_fn(labels, preds)
            
        grad = tape.gradient(d_loss, self.discriminator.trainable_weights)
        self.d_optimizer.apply_gradients(zip(grad, self.discriminator.trainable_weights))
        
        random_input = tf.random.normal(shape=(batch_size, self.latent_dim))
        gen_labels = tf.zeros((batch_size, 1))
        
        with tf.GradientTape() as tape:
            preds = self.discriminator(self.generator(random_input))
            g_loss = self.loss_fn(gen_labels, preds)
            
        grad = tape.gradient(g_loss, self.generator.trainable_weights)
        self.g_optimizer.apply_gradients(zip(grad, self.generator.trainable_weights))

        return {'d_loss': d_loss, 'g_loss': g_loss}  

In [90]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = np.expand_dims(x_train, -1)
x_train = (x_train - 127.5) / 127.5
y_train = y_train
y_test = y_test

In [84]:
#idx_train = np.where(y_train == 1)[0]
#x_train = x_train[idx_train]

#idx_test = np.where(y_test == 1)[0]
#x_test = x_test[idx_test]

In [117]:
def loss_fn(labels, preds):
    return losses.BinaryCrossentropy(from_logits=True)(labels, preds)

d_optimizer = optimizers.Adam(lr=0.001)
g_optimizer = optimizers.Adam(lr=0.001)

model = GAN(latent_dim=100)

model.compile(d_optimizer, g_optimizer, loss_fn)
model.fit(x_train, epochs=1)

4680/4680 [==============================] - 588s 126ms/step - d_loss: 0.4911 - g_loss: 1.3122
